### Import libraries

In [24]:
# Import necessary libraries
import os

# Import necessary libraries for data processing
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Import necessary libraries for model
from keras.callbacks import Callback
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from keras.metrics import Precision, Recall, F1Score, AUC

# Import necessary libraries for evaluation
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


## Directories

In [25]:
market_data_1_dir = 'market_data_A_0.csv'
market_data_2_dir = 'market_data_A_1.csv'

trade_data_dir = 'trade_data__A.csv'

Save_model_dir = 'LSTM_model'

# Data Preparation

### Load and merge data

In [26]:
# Load the data from a CSV file
market_data_1 = pd.read_csv(market_data_1_dir)
market_data_2 = pd.read_csv(market_data_2_dir)
market_data = pd.concat([market_data_1, market_data_2], ignore_index = True)

# Load the trade data from a CSV file
trade_data = pd.read_csv(trade_data_dir)

# Handle the null values
market_data.dropna(subset=['timestamp'], inplace=True)
trade_data.dropna(subset=['timestamp'], inplace=True)

# Convert the 'timestamp' column into datetime
market_data['timestamp'] = pd.to_datetime(market_data['timestamp'])
trade_data['timestamp'] = pd.to_datetime(trade_data['timestamp'])

# merge the data on the nearest timestamp
merged_data = pd.merge_asof(
    market_data.sort_values('timestamp'),
    trade_data.sort_values('timestamp'),
    by='timestamp',
    direction='nearest')

<ipython-input-26-cee42e615d8a>:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  market_data['timestamp'] = pd.to_datetime(market_data['timestamp'])
<ipython-input-26-cee42e615d8a>:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trade_data['timestamp'] = pd.to_datetime(trade_data['timestamp'])


### Feature Engineering

In [27]:
# Calculate the spread between the market and trade data
merged_data['spread'] = merged_data['askPrice'] - merged_data['bidPrice']

# Add a moving average of the spread
merged_data['bidPrice_ma'] = merged_data['bidPrice'].rolling(window=5).mean()
merged_data['askPrice_ma'] = merged_data['askPrice'].rolling(window=5).mean()

# Fill any remaining NANS by backfilling
merged_data.fillna(method='bfill', inplace=True)

<ipython-input-27-70c1d597eccc>:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='bfill', inplace=True)
<ipython-input-27-70c1d597eccc>:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_data.fillna(method='bfill', inplace=True)


### Define the feature set and target set

In [28]:
# Define the feature and it's fitted scaller
feature_scaler = MinMaxScaler()
features = [
    'bidVolume',
    'bidPrice',
    'askVolume',
    'askPrice',
    'spread',
    'bidPrice_ma',
    'askPrice_ma'
    ]
feature_scaler.fit(merged_data[features])

# Defien the target scaller adn fit it
target_scaler = MinMaxScaler()
target_scaler.fit(merged_data[['bidPrice']])

MinMaxScaler()

### Create a scaled array

In [30]:
x_data = merged_data[features].values
y_data = merged_data[['bidPrice']].values

# Scale the data
x_data_scaled = feature_scaler.transform(x_data)
y_data_scaled = target_scaler.transform(y_data)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


### Define input and output

In [31]:
# Define timesteps
timesteps = 10

# Prepare input (x) and output (y)
x = []
y = []

for i in range(len(merged_data) - timesteps):
    x.append(x_data_scaled[i:i + timesteps])
    y.append(y_data_scaled[i + timesteps])

x = np.array(x)
y = np.array(y)

### Split data

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Implementation of Model

### Define Model class

In [33]:
# reshape the input data
input_shape = (x_train.shape[1], x_train.shape[2])

# define the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=input_shape, return_sequences=True))
model.add(Dense(1, activation='linear')) # TODO:Adjust the activation function to sigmoid if needed

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Compile model

In [34]:
# compile the model for training
model.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['mae']
    ) # TODO:Adjust the loss function to binary_crossentropy if needed

### Train and validate model

In [ ]:

# # train the model
# model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test))


# Custom callback to save and print metrics at each epoch
class EpochLogger(Callback):
    def __init__(self):
        super().__init__()
        self.epochs = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        # Print the metrics for the current epoch
        print(f"Epoch {epoch + 1}: {logs}")

        # Save metrics in a list for later use
        self.epochs.append({"epoch": epoch + 1, **logs})

# Instantiate the custom callback
epoch_logger = EpochLogger()

# Train the model and use the custom callback
history = model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_data=(x_test, y_test),
    callbacks=[epoch_logger]
)

# Save the epoch logs to a CSV file
epoch_logs_df = pd.DataFrame(epoch_logger.epochs)
epoch_logs_df.to_csv('epoch_logs.csv', index=False)

# Print the saved dataframe
print(epoch_logs_df)

Epoch 1/20
1938/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.0055 - mae: 0.0283

### Test model

In [23]:
# Predict on the scaled test data
y_pred_scaled = model.predict(x_test)

# Apply inverse transformation to scale back to the original range and inverse transform y_test
y_pred_original = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test)

print("Original Test Values (unscaled):", y_test_original.ravel()[:10])
print("Predicted Values (unscaled):", y_pred_original.ravel()[:10])

625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


ValueError: non-broadcastable output operand with shape (199980,1) doesn't match the broadcast shape (199980,7)

## Other resources code

### Load model

In [ ]:
loaded_model = load_model(Save_model_dir)

### Save model

In [ ]:
# save in tensoflow model
model.save(Save_model_dir)